# Лабораторная работа №6

## Цель работы
Освоить практические навыки создания веб-API с использованием FastAPI, научиться создавать различные типы эндпоинтов, работать с параметрами запросов, валидировать данные и интегрировать API с базой данных SQLite.

## Предварительные требования
```bash
pip install fastapi uvicorn pydantic
```

## Задание

### Часть 1: Создание базового API для управления задачами (30 баллов)

Создайте приложение `main.py` для управления списком задач (To-Do List).

**Требуемые эндпоинты:**

1. **GET /** — Приветственное сообщение
   ```json
   {"message": "Welcome to Task Management API", "version": "1.0"}
   ```

2. **GET /tasks** — Получить все задачи
   - Query-параметры: `completed` (boolean, необязательный) — фильтр по статусу выполнения
   - Пример: `/tasks?completed=true`

3. **GET /tasks/{task_id}** — Получить задачу по ID
   - Параметр пути: `task_id` (int)
   - Если задача не найдена, вернуть ошибку 404

4. **POST /tasks** — Создать новую задачу
   - Тело запроса (JSON):
   ```json
   {
     "title": "Выучить FastAPI",
     "description": "Прочитать документацию и сделать лабораторную",
     "priority": "high"
   }
   ```

**Модель данных Pydantic:**
```python
from pydantic import BaseModel, Field
from typing import Optional

class Task(BaseModel):
    title: str = Field(..., min_length=1, max_length=100)
    description: Optional[str] = None
    priority: str = Field(default="medium", pattern="^(low|medium|high)$")
    completed: bool = False
```

**Требования:**
- Храните задачи в списке (на данном этапе без базы данных)
- Каждой задаче автоматически присваивайте уникальный ID
- Используйте правильные HTTP-статус коды (200, 201, 404)

### Часть 2: Расширенные операции CRUD (30 баллов)

Добавьте следующие эндпоинты:

1. **PUT /tasks/{task_id}** — Полностью обновить задачу
   - Все поля обязательны
   - Вернуть обновлённую задачу

2. **PATCH /tasks/{task_id}/complete** — Отметить задачу как выполненную
   - Изменяет только поле `completed` на `true`

3. **DELETE /tasks/{task_id}** — Удалить задачу
   - Вернуть сообщение об успешном удалении

4. **GET /tasks/search** — Поиск задач
   - Query-параметры:
     - `keyword` (str) — поиск в названии и описании
     - `priority` (str) — фильтр по приоритету

5. **GET /tasks/stats** — Статистика по задачам
   - Вернуть:
     ```json
     {
       "total": 10,
       "completed": 3,
       "pending": 7,
       "by_priority": {
         "high": 2,
         "medium": 5,
         "low": 3
       }
     }
     ```

**Требования:**
- Используйте правильные HTTP-методы для каждой операции
- Добавьте обработку ошибок (задача не найдена, некорректные данные)
- Все эндпоинты должны возвращать понятные сообщения

### Часть 3: Интеграция с базой данных SQLite (40 баллов)

Замените хранение данных в списке на базу данных SQLite.

**Требования:**

1. Создайте модуль `database.py` с функциями:
   - `init_db()` — создание таблицы tasks
   - `add_task(task)` — добавление задачи в БД
   - `get_all_tasks(completed=None)` — получение всех задач с фильтром
   - `get_task_by_id(task_id)` — получение задачи по ID
   - `update_task(task_id, task_data)` — обновление задачи
   - `delete_task(task_id)` — удаление задачи
   - `get_statistics()` — получение статистики

2. Структура таблицы `tasks`:
   ```sql
   CREATE TABLE tasks (
       id INTEGER PRIMARY KEY AUTOINCREMENT,
       title TEXT NOT NULL,
       description TEXT,
       priority TEXT NOT NULL,
       completed INTEGER DEFAULT 0,
       created_at TEXT DEFAULT CURRENT_TIMESTAMP
   )
   ```

3. Модифицируйте все эндпоинты для работы с БД вместо списка

4. Добавьте поле `created_at` в модель ответа

**Что проверяется:**
- Правильная интеграция FastAPI с SQLite
- Использование параметризованных запросов (защита от SQL Injection)
- Обработка ошибок базы данных

## Требования к оформлению

1. **Структура проекта:**
   ```
   lab4/
   ├── main.py          # FastAPI приложение
   ├── database.py      # Работа с SQLite
   ├── models.py        # Pydantic модели
   ├── tasks.db         # База данных (создаётся автоматически)
   └── README.md        # Описание API и инструкция по запуску
   ```

2. **Код должен быть читаемым:**
   - Используйте осмысленные имена переменных
   - Добавьте комментарии к сложным участкам
   - Следуйте PEP 8

3. **Документация в README.md:**
   - Описание проекта
   - Инструкция по установке
   - Список всех эндпоинтов с примерами
   - Примеры запросов curl или Python requests

## Пример использования API

```python
import requests

BASE_URL = "http://127.0.0.1:8000"

# Создать задачу
response = requests.post(f"{BASE_URL}/tasks", json={
    "title": "Изучить FastAPI",
    "description": "Пройти лабораторную работу",
    "priority": "high"
})
print(response.json())

# Получить все задачи
response = requests.get(f"{BASE_URL}/tasks")
print(response.json())

# Отметить задачу как выполненную
response = requests.patch(f"{BASE_URL}/tasks/1/complete")
print(response.json())
```

## Критерии оценки

- **Часть 1** (30 баллов): Базовые эндпоинты GET/POST с валидацией
- **Часть 2** (30 баллов): Полный CRUD + поиск и статистика
- **Часть 3** (40 баллов): Интеграция с SQLite

**Максимальный балл: 100**